<h1>IFLA WLIC - 2022</h1>
<h2><span style="font-size:11pt;font-family:Arial;color:#000000;background-color:transparent;font-weight:700;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">From text to data inside bibliographic records. Entity recognition and entity linking of contributors and their roles from statements of responsibility</span></h2>

<h2>Pip installs - Necessary libraries</h2>

In [111]:
#!pip install spacy
#!pip install fuzzywuzzy
#!pip install python-Levenshtein
!pip install texthero

  Using cached texthero-1.1.0-py3-none-any.whl (24 kB)
  Using cached wordcloud-1.8.1.tar.gz (220 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached spacy-2.3.7-cp39-cp39-win_amd64.whl (9.4 MB)
  Using cached gensim-3.8.3.tar.gz (23.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached plotly-5.8.0-py2.py3-none-any.whl (15.2 MB)


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [17 lines of output]
  running bdist_wheel
  running build
  running build_py
  running egg_info
  writing gensim.egg-info\PKG-INFO
  writing dependency_links to gensim.egg-info\dependency_links.txt
  writing requirements to gensim.egg-info\requires.txt
  writing top-level names to gensim.egg-info\top_level.txt
  reading manifest file 'gensim.egg-info\SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'COPYING'
  running build_ext
  building 'gensim.models.word2vec_inner' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gensim
  error: subprocess-exited-with-error
  
  python setup.py bd

  Running setup.py clean for gensim
  Running setup.py clean for wordcloud
Failed to build gensim wordcloud
  Running setup.py install for gensim: started
  Running setup.py install for gensim: finished with status 'error'



  running bdist_wheel
  running build
  running build_py
  UPDATING build\lib.win-amd64-3.9\wordcloud/_version.py
  set build\lib.win-amd64-3.9\wordcloud/_version.py to '1.8.1'
  running build_ext
  building 'wordcloud.query_integral_image' extension
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\zaragoza\AppData\Local\Temp\pip-install-pfdd2k7k\wordcloud_5dc359b2fd4744b4bf2df2adf56fdb72\setup.py", line 9, in <module>
      setup(
    File "C:\Users\zaragoza\AppData\Roaming\Python\Python39\site-packages\setuptools\__init__.py", line 155, in setup
      return distutils.core.setup(**attrs)
    File "C:\Users\zaragoza\AppData\Roaming\Python\Python39\site-packages\setuptools\_distutils\core.py", line 148, in setup
      return run_commands(dist)
    File "C:\Users\zaragoza\AppData\Roaming\Python\Python39\site-packages\setuptools\_distutils\core.py", line 163, in run_commands
      di

In [ ]:
!pip uninstall pip

In [ ]:
!pip install pip

<h2>Imports - Necessary libraries</h2>

In [101]:
# Formats
import csv
import unidecode
import pandas as pd

# Functions
import re
from fuzzywuzzy import fuzz

# Models
import spacy
from spacy import displacy

#Data pre-treatment
from sklearn import preprocessing

<h2>Demonstration</h2>

<h3>NER (Named Entity Recognition) Model</h3>

In [44]:
model = spacy.load("./Model_NER_PERS_FONCT_10K")
doc = model("écrit par Zac Thompson et Emily Horn, dessiné par Alberto Albuquerque")
displacy.render(doc, style="ent")

<h3>Creating training sets</h3>

In [80]:
def name_match(name1,name2):
    titreCivilite=0
    score=0
    name1=name1.replace("-"," ")
    name1 = re.sub(r"(.[.])()(.[.])",r"\1 \3", name1)
    name2=name2.replace("-"," ")
    name_components_1 = name1.split(" ")
    name_components_2 = name2.split(" ")
    nameComponent1Match={}
    nameComponent2Match={}
    for index in range(len(name_components_1)):
        name_component1=unidecode.unidecode(name_components_1[index]).lower()
        ##TITLE##
        if(index==0 and (name_component1 in ["m.","mme.","dr."])):
            titreCivilite=1
            score+=1
        ##ABREVIATIION##
        elif(len(name_component1)<3 or ("." in name_component1)):
            for index2 in range(len(name_components_2)):
                name_component2=unidecode.unidecode(name_components_2[index2]).lower()
                if(name_component2!=''):
                    match=True
                    for i in range(0,len(name_component1)):
                        if(i<len(name_component2)and name_component1[i]!=name_component2[i] and name_component1[i]!="."):
                            match=False
                    if(match):
                        if(name_component1 not in nameComponent1Match.keys()):
                            nameComponent1Match[name_component1]=1
                        else:
                            nameComponent1Match[name_component1]+=1
                        if(name_component2 not in nameComponent2Match.keys()):
                            nameComponent2Match[name_component2]=1
                        else:
                            nameComponent2Match[name_component2]+=1
        ##Complete Name##
        else:
            for index2 in range(len(name_components_2)):
                    name_component2=unidecode.unidecode(name_components_2[index2]).lower()
                    ratio = fuzz.ratio(name_component1,name_component2)
                    if(name_component2!=''):
                        if(name_component1==name_component2 or (ratio>=80)):
                            if(name_component1 not in nameComponent1Match.keys()):
                                alignment=''
                                for key in nameComponent1Match.keys():
                                    if(fuzz.ratio(name_component1,key)>=80):
                                        alignment=key
                                if(alignment==''):
                                    nameComponent1Match[name_component1]=1
                                else:
                                    nameComponent1Match[alignment]+=1
                            else:
                                nameComponent1Match[name_component1]+=1
                            if(name_component2 not in nameComponent2Match.keys()):
                                alignment=''
                                for key in nameComponent2Match.keys():
                                    if(fuzz.ratio(name_component2,key)>=80):
                                        alignment=key
                                if(alignment==''):
                                    nameComponent2Match[name_component2]=1
                                else:
                                    nameComponent2Match[alignment]+=1
                            else:
                                nameComponent2Match[name_component2]+=1                        
    for key in nameComponent1Match.keys():
        score+=nameComponent1Match[key]    
    
    #print("Matching between "+name1+" and "+name2+" has a score of "+str(score))
    if(score==len(name_components_1) and (len(nameComponent2Match.keys())+titreCivilite)==(len(name_components_1))):
        return True
    else:
        return False
    
def create_trainingSet(load_name,save_name):
    model = spacy.load("./Model_NER_PERS_FONCT_10K")
    rows=[]
    file = open("./data/"+load_name,mode='r',encoding="utf8")
    csvreader = csv.reader(file, delimiter=",")
    header = next(csvreader)
    dicoLabelAmt={}
    #############For ever bibliographical record entry in the csv file#############
    for row in csvreader:
        ppn=row[1]
        bibliographicalRecord = row[2]
        accessPoints = row[3]
        documentSupports = row[4]
        ppnLinks = row[5]
        textualRessources = row[6]
        mentions = bibliographicalRecord.split(" / ",1)
        ###############################Creating table of supports#############################
        supports = documentSupports.split(",")
        ######################################################################################
        ##########################Creating access points dictionary##########################
        dictionnaireAccessPoints = {}
        codes=[]
        accessPoint = accessPoints.split(" ")
        name=""
        for token in accessPoint :
            if(token.isnumeric() and len(name)==0):
                codes.append(token)
            elif(token.isnumeric()):
                #Checking the person is associate to only a single UNIMARC code function to avoi ambiguity
                if (len(name)>0 and len(codes)==1):
                    dictionnaireAccessPoints[name]=codes
                codes=[]
                name=""
                codes.append(token)
            elif(len(name)==0):
                name+=token
            else :
                name+=" "
                name+=token
        #Checking the person is associate to only a single UNIMARC code function to avoi ambiguity
        if (len(name)>0 and len(codes)==1):
            dictionnaireAccessPoints[name]=codes
        ########################################################################################
        ###################################Feature 'Thesis'#####################################
        isThesis = False
        for link in ppnLinks.split(','):
            if(link=="027253139"):
                isThesis=True
        for ressource in textualRessources.split(','):
            if(('m' in ressource[4:8])or('v' in ressource[4:8])):
                isThesis=True
        ########################################################################################
        ####################################Extracting LABEL####################################
        if(len(mentions)>=2):
            ###Position of the mention###
            position=0
            ###Iterating through the mention of responsibilities###
            for mention in mentions[1].split(" ; "):
                dictPersonFunct={}
                doc = model(mention)
                ###Pairing function keywords to their appropriate persons###
                persons=[]
                keywords=[]
                currentFunct=''
                switch=0
                for ent in doc.ents:
                    if(currentFunct==""):
                        currentFunct=ent.label_
                    if(currentFunct!=ent.label_):
                        currentFunct=ent.label_
                        switch+=1
                    if(switch%2==0 and switch>0):
                        for person in persons:
                            if(person in dictPersonFunct.keys()):
                                dictPersonFunct[person].append(keywords)
                            else:
                                dictPersonFunct[person]=keywords
                        persons=[]
                        keywords=[]
                    if(ent.label_== "PER"):
                        persons.append(ent)
                    if(ent.label_ == "FONCT"):
                        keywords.append(ent)
                for person in persons:
                    if(person in dictPersonFunct.keys()):
                        dictPersonFunct[person].append(keywords)
                    else:
                        dictPersonFunct[person]=keywords
                ############################################################
                for person in dictPersonFunct.keys():
                    for key in dictionnaireAccessPoints.keys():
                        if(name_match(person.text, key)):
                            ###Counting amount of entries per function###
                            if(dictionnaireAccessPoints[key][0] in dicoLabelAmt.keys()):
                                dicoLabelAmt[dictionnaireAccessPoints[key][0]]+=1
                            else:
                                dicoLabelAmt[dictionnaireAccessPoints[key][0]]=1
                            ############################################
                            rows.append([ppn,person.text,dictPersonFunct[person],position,supports,isThesis,dictionnaireAccessPoints[key][0]])
                ###incrementing position of the mention###
                position+=1
    ########################################################################################
    ##################################Writing training dataset##############################
    file2 = open("./data/TRAINING_PPN_PERSON_Keywords_Position_DocSupport_Thesis"+save_name+".csv",mode='w',encoding="utf8",newline='')
    writer = csv.writer(file2)
    writer.writerow(["PPN","Person","Keywords","Position","Document Support","Thesis","Label"])
    for row in rows:
        writer.writerow(row)
    print("There are "+str(len(rows))+" entries")
    print("Final count of training entries per function :")
    for key in dicoLabelAmt.keys():
        print("Function "+str(key)+" has "+str(dicoLabelAmt[key])+" entries.")

In [81]:
true_name1=["F. Nietzsche","Alberto Albuquerque","M. Zaragoza","Mme. Zaragoza","Thomâs Zargoza","Mircel Frontignan","J-C Van Damme","Th Maximilien","J.L. Istin"]
true_name2=["Friedrich Nietzsche","Alberto Jiménez Albuquerque","Thomas Zaragoza","Zaragoza Thomas","Thomas Zaragoza","Marcel Frontagnan","Jean-Claude Van Damme","Théophile Maximilien","Jean-Luc Istin"]
false_name1=["G. Nietzsche","Frank Nietzsche","Z. Zaragoza","Marcel Frontignan"]
false_name2=["Friedrich Nietzsche","Friedrich Nietzsche","Thomas Zaragoza","Marco Frontignan"]
print("Matching between these should be accepted")
print(" ")
for x in range(len(true_name1)):
    print(true_name1[x]+" and "+true_name2[x]+" are matching: "+str(name_match(true_name1[x],true_name2[x])))
    Token_Set_Ratio = fuzz.token_set_ratio(true_name1[x],true_name2[x])
    print("Token_Set_Ratio is :"+str(Token_Set_Ratio))
print(" ")
print("========================================")
print(" ")
print("Matching between these should be refused")
print(" ")
for x in range(len(false_name1)):
    print(false_name1[x]+" and "+false_name2[x]+" are matching : "+str(name_match(false_name1[x],false_name2[x])))
    Token_Set_Ratio = fuzz.token_set_ratio(false_name1[x],false_name2[x])

Matching between these should be accepted
 
F. Nietzsche and Friedrich Nietzsche are matching: True
Token_Set_Ratio is :90
Alberto Albuquerque and Alberto Jiménez Albuquerque are matching: True
Token_Set_Ratio is :100
M. Zaragoza and Thomas Zaragoza are matching: True
Token_Set_Ratio is :89
Mme. Zaragoza and Zaragoza Thomas are matching: True
Token_Set_Ratio is :80
Thomâs Zargoza and Thomas Zaragoza are matching: True
Token_Set_Ratio is :93
Mircel Frontignan and Marcel Frontagnan are matching: True
Token_Set_Ratio is :88
J-C Van Damme and Jean-Claude Van Damme are matching: True
Token_Set_Ratio is :82
Th Maximilien and Théophile Maximilien are matching: True
Token_Set_Ratio is :87
J.L. Istin and Jean-Luc Istin are matching: True
Token_Set_Ratio is :78
 
 
Matching between these should be refused
 
G. Nietzsche and Friedrich Nietzsche are matching : False
Frank Nietzsche and Friedrich Nietzsche are matching : False
Z. Zaragoza and Thomas Zaragoza are matching : False
Marcel Frontignan a

In [82]:
create_trainingSet("ID_PPN_Mentions_AP_DocSup_PPNLink_TextualRessources_TEST.csv","")

There are 8 entries
Final count of training entries per function :
Function 070 has 2 entries.
Function 440 has 1 entries.
Function 410 has 1 entries.
Function 730 has 1 entries.
Function 005 has 3 entries.


<h2>Loading and encoding training set</h2>

In [102]:
def encodage_données(dataframe,featuresBinaires, featuresOrdinaux, featuresNominaux, featuresTextes):
    encodedDataframe = dataframe.copy()
    ###Relocating target##
    labelColumn = encodedDataframe["Label"] 
    del encodedDataframe["Label"]
    ######################
    clean_pipeline = [#preprocessing.fillna,
                   preprocessing.lowercase,
                   preprocessing.remove_whitespace,
                   preprocessing.remove_diacritics
                   #preprocessing.remove_brackets
                  ]
    ######################
    binaryEncoders=[]
    ordinalEncoders=[]
    nominalEncoders=[]
    textEncoders=[]
    ###Categorical Features###
    for binaryFeature in featuresBinaires:
        print("Encoding Binary Feature : "+str(binaryFeature))
        label_encoder = LabelEncoder()
        label_encoder.fit(encodedDataframe[binaryFeature])
        encodedDataframe[binaryFeature]=label_encoder.transform(dataframe[binaryFeature])
        binaryEncoders.append(label_encoder)
    for ordinalFeature in featuresOrdinaux:
        pass
        #IMPLEMENT
    for nominalFeature in featuresNominaux:
        print("Encoding Nominal Feature : "+str(nominalFeature))
        encodedDataframe = pd.get_dummies(encodedDataframe, prefix = [nominalFeature], columns = [nominalFeature])
        nominalEncoders.append(nominalFeature)
    ###Text Features###
    for textFeature in featuresTextes:
        print("Encoding Text Feature : "+str(textFeature))
        #######CLEAN UP#########
        encodedDataframe[textFeature] = [n.replace('[','') for n in encodedDataframe[textFeature]]
        encodedDataframe[textFeature] = [n.replace(']','') for n in encodedDataframe[textFeature]]
        encodedDataframe[textFeature] = hero.clean(encodedDataframe[textFeature], clean_pipeline)
        ########################
        ###Creating TF-IDF df###
        encodedDataframe[textFeature]=hero.tfidf(encodedDataframe[textFeature], max_features=3000)
        allwordsAmt=len(encodedDataframe[textFeature][0])
        lsts=list(encodedDataframe[textFeature])
        column_names=[]
        for i in range(0,allwordsAmt):
            column_names.append(str(textFeature)+"_"+str(i))
        dataframeTmp = pd.DataFrame(lsts, columns =column_names, dtype = float)
        ########################
        del encodedDataframe[textFeature]
        encodedDataframe = pd.concat([encodedDataframe.reset_index(drop=True),dataframeTmp.reset_index(drop=True)], axis=1)
    ###Relocating target###
    #encodedDataframe["Label"]=labelColumn
    #######################
    return encodedDataframe,labelColumn,binaryEncoders, ordinalEncoders, nominalEncoders, textEncoders

In [90]:
df = pd.read_csv("./data/TRAINING_Classes_13_Sample_1000.csv")

In [104]:
df

,PPN,Personne,Position,Keywords,Document support,Thesis,Label
0,262419734,Béatrix Lot,0,[],"['txt', 'sti']",False,70
1,262419610,Amadou Amal,0,[],['txt'],False,70
2,262419572,Séverine Vidal,0,[],['txt'],False,70
3,262419467,Fabrice Amedeo,0,[],['txt'],False,70
4,262419378,Bruno d'Halluin,0,[],['txt'],False,70
...,...,...,...,...,...,...,...
12995,249884380,Anna Gavalda,1,"[aut., adapté]",['tdi'],False,100
12996,24987900X,Maurice Blanchot,3,[aut.],['tdi'],False,100
12997,24987900X,Edgar Allan Poe,3,[aut.],['tdi'],False,100
12998,249876671,Joseph Conrad,1,[aut.],['tdi'],False,100


In [105]:
encodedDataframe,targetDataframe,binaryEncoders, ordinalEncoders, nominalEncoders, textEncoders = encodage_données(df,["Thesis"],[],[],["Document support","Keywords"])

AttributeError: module 'sklearn.preprocessing' has no attribute 'lowercase'